In [1]:
import os
os.system('export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:/datasets/home/22/422/ee276cao/.mujoco/mjpro150/bin')

0

In [1]:
import math
import random

import gym
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Normal

import os
logdir='./DDPG/'
logging_interval = 100
animate_interval = logging_interval * 5

In [2]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
import gym.spaces
import sys
sys.path.append('/datasets/home/22/422/ee276cao/.mujoco/mjpro150/bin')

<h2>Use CUDA</h2>

In [4]:
use_cuda = torch.cuda.is_available()
device   = torch.device("cuda" if use_cuda else "cpu")

<h2>Create Environments</h2>

In [7]:
from common.multiprocessing_env import SubprocVecEnv

#num_envs = 16
num_envs = 16
# env_name = "Pendulum-v0" 
# env_name = 'HalfCheetah-v2' 
# env_name = 'SpaceInvaders-v0'
# env_name = 'HandManipulateBlock-v0'
env_name = 'Ant-v1'

def make_env():
    def _thunk():
        env = gym.make(env_name)
        return env

    return _thunk
# 
envs = [make_env() for i in range(num_envs)]
envs = SubprocVecEnv(envs)
# print(os.environ['LD_LIBRARY_PATH'])
env = gym.make(env_name)

[2018-06-02 14:22:03,925] Making new env: Ant-v1
[2018-06-02 14:22:03,933] Making new env: Ant-v1
[2018-06-02 14:22:03,939] Making new env: Ant-v1
[2018-06-02 14:22:03,946] Making new env: Ant-v1
[2018-06-02 14:22:03,951] Making new env: Ant-v1
[2018-06-02 14:22:03,957] Making new env: Ant-v1
[2018-06-02 14:22:03,963] Making new env: Ant-v1
[2018-06-02 14:22:03,969] Making new env: Ant-v1
[2018-06-02 14:22:03,976] Making new env: Ant-v1
[2018-06-02 14:22:03,983] Making new env: Ant-v1
[2018-06-02 14:22:03,991] Making new env: Ant-v1
[2018-06-02 14:22:03,996] Making new env: Ant-v1
[2018-06-02 14:22:04,001] Making new env: Ant-v1
[2018-06-02 14:22:04,007] Making new env: Ant-v1
[2018-06-02 14:22:04,013] Making new env: Ant-v1
[2018-06-02 14:22:04,019] Making new env: Ant-v1
[2018-06-02 14:22:04,018] Making new env: Ant-v1


In [8]:
# import gym
# env = gym.make('SpaceInvaders-v0')
# env.reset()
# env.render()

# for _ in range(2):
#     observation = env.render()
#     for _ in range(3):
#         env.render()
#         print (observation)
#         action = env.action_space.sample()
#         observation, reward, done, info = env.step(action)



In [21]:
# import sys
# sys.path.append("/datasets/home/22/422/ee276cao/.mujoco/mjpro150/bin") 

In [22]:
# import gym
# env = gym.make('HandManipulateBlock-v0')
# env.reset()
# env.render()

In [9]:
VISUALIZE = True
SEED = 0 
MAX_PATH_LENGTH = 500 
NUM_EPISODES = 12000
if VISUALIZE:
    if not os.path.exists(logdir):
        os.mkdir(logdir)
    env = gym.wrappers.Monitor(env, logdir, force=True, video_callable=lambda episode_id: episode_id%logging_interval==0)

#Need to look through code more before setting this. 
#env._max_episode_steps = MAX_PATH_LENGTH


[2018-06-02 14:22:08,585] Clearing 2 monitor files from previous run (because force=True was provided)


<h2>Neural Network</h2>

In [10]:
def init_weights(m):
    if isinstance(m, nn.Linear):
        nn.init.normal_(m.weight, mean=0., std=0.1)
        nn.init.constant_(m.bias, 0.1)

class Flatten(nn.Module):
    def forward(self, input):
        return input.view(input.size(0), -1)

class ActorCritic(nn.Module):
    def __init__(self, num_inputs, num_outputs, hidden_size, std=0.0):
        super(ActorCritic, self).__init__()

#        self.critic = nn.Sequential(
#            nn.Linear(num_inputs, hidden_size),
#            nn.ReLU(),
#            nn.Linear(hidden_size, 1)
#        )
        self.critic = nn.Sequential(
            nn.Conv2d(in_channels=num_inputs[2], out_channels=32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
            nn.ReLU(),
            Flatten(),
            nn.Linear(7*7*64, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_outputs)
         )
    
        print(self.critic)
        
        self.actor = nn.Sequential(
            nn.Conv2d(in_channels=num_inputs[2], out_channels=32, kernel_size=8, stride=4),
            nn.ReLU(),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=4, stride=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1),
            nn.ReLU(),
            Flatten(),
            nn.Linear(7*7*64, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, num_outputs)
        )
        self.log_std = nn.Parameter(torch.ones(1, num_outputs) * std)
        print ("self.long_std: ", self.log_std)
        
        self.apply(init_weights)
        
    def forward(self, x):
        value = self.critic(x)
        mu    = self.actor(x)
        std   = self.log_std.exp().expand_as(mu)
        dist  = Normal(mu, std)
        return dist, value

In [11]:
def plot(frame_idx, rewards):
    clear_output(True)
    plt.figure(figsize=(20,5))
    plt.subplot(131)
    plt.title('frame %s. reward: %s' % (frame_idx, rewards[-1]))
    plt.plot(rewards)
    plt.show()
    
def test_env(vis=False):
    state = env.reset()
    if vis: env.render()
    done = False
    total_reward = 0
    while not done:
        state = torch.FloatTensor(state).unsqueeze(0).to(device)
        dist, _ = model(state)
        next_state, reward, done, _ = env.step(dist.sample().cpu().numpy()[0])
        state = next_state
        if vis: env.render()
        total_reward += reward
    return total_reward

In [12]:
def createStandardDeviation(vis=False):
    all_std = [testEnv(True) for x in range(100)]
    sd = np.std(all_std)
    print ("all_std: ", all_std)
    print ("sd: ", sd)
    return sd

<h1>High-Dimensional Continuous Control Using Generalized Advantage Estimation</h1>
<h3><a href="https://arxiv.org/abs/1506.02438">Arxiv</a></h3>

In [13]:
def compute_gae(next_value, rewards, masks, values, gamma=0.99, tau=0.95):
    values = values + [next_value]
    gae = 0
    returns = []
    for step in reversed(range(len(rewards))):
        delta = rewards[step] + gamma * values[step + 1] * masks[step] - values[step]
        gae = delta + gamma * tau * masks[step] * gae
        returns.insert(0, gae + values[step])
    return returns

In [14]:
# num_inputs_x  = envs.observation_space.shape[0]
# num_inputs_y  = envs.observation_space.shape[1]
# num_inputs_c  = envs.observation_space.shape[2]
# num_inputs = (num_inputs_x, num_inputs_y, num_inputs_c)
# # num_outputs = envs.action_space.shape[0]
# num_outputs = env.action_space.n

# #Hyper params
# num_stacked_frames = 4
# hidden_size = 512
# lr          = 3e-4
# num_steps   = 20

# print ("numInputs: {} x {} x {}".format(num_inputs_x, num_inputs_y, num_inputs_c))
# print ('numOutputs: ', num_outputs)
# print ("hidden_size ", hidden_size)

# model = ActorCritic((84,84,num_stacked_frames), num_outputs, hidden_size).to(device)
# optimizer = optim.Adam(model.parameters())

IndexError: tuple index out of range

In [17]:
# print(envs.action_space.shape[0])
# print(envs.observation_space.__dict__)
# print(envs.observation_space.spaces['observation'])

# num_inputs  = envs.observation_space.spaces['observation'].shape[0]
# num_outputs = envs.action_space.shape[0]

num_inputs = envs.observation_space.shape[0]
num_outputs = envs.action_space.shape[0]

#Hyper params:
hidden_size = 256
lr          = 3e-4
num_steps   = 20

model = ActorCritic(num_inputs, num_outputs, hidden_size).to(device)
optimizer = optim.Adam(model.parameters())


TypeError: 'int' object is not subscriptable

In [28]:
# print(env.action_space)
# space = env.action_space
# print(env.action_space.n)
# # print (envs.action_space.shape)

In [29]:
max_frames   = 100000
frame_idx    = 0
test_rewards = []

In [30]:
import numpy as np
from PIL import Image

def process_state(frame):
    # rid of extra dim
    img = np.reshape(frame, [210, 160, 3])
    # convert to b/w
    img = img[:, :, 0] * 0.299 + img[:, :, 1] * 0.587 + img[:, :, 2] * 0.114
    # resize
    img = Image.fromarray(img)
    resized_screen = img.resize((84, 84), Image.BILINEAR)
    resized_screen = np.array(resized_screen)
    return np.reshape(resized_screen, [1,84,84])

In [31]:
state = envs.reset()

while frame_idx < max_frames:

    log_probs = []
    values    = []
    rewards   = []
    masks     = []
    entropy = 0
    
    frames = np.zeros([num_stacked_frames, 84, 84])

    for _ in range(num_steps):
        state = torch.FloatTensor(state).to(device)
        print(state.shape)
        # keep num_stacked_frames
        frame1 = frames[1,:,:].reshape([1,84,84])
        frame2 = frames[2,:,:].reshape([1,84,84])
        frame3 = frames[3,:,:].reshape([1,84,84])
        frame4 = process_state(state.permute(0,3,1,2).cpu().numpy())
        print(frame1.shape, frame2.shape, frame3.shape, frame4.shape)
        frames = np.concatenate((frame1, frame2, frame3, frame4))
        dist, value = model(torch.FloatTensor(frames.reshape([1,4,84,84])).to(device))
        print("dist", dist)

        action = dist.sample()
        next_state, reward, done, _ = envs.step(action.cpu().numpy())

        log_prob = dist.log_prob(action)
        entropy += dist.entropy().mean()
        
        log_probs.append(log_prob)
        values.append(value)
        rewards.append(torch.FloatTensor(reward).unsqueeze(1).to(device))
        masks.append(torch.FloatTensor(1 - done).unsqueeze(1).to(device))
        
        state = next_state
        frame_idx += 1
        
        if frame_idx % 1000 == 0:
            test_rewards.append(np.mean([test_env() for _ in range(10)]))
            plot(frame_idx, test_rewards)
            
    next_state = torch.FloatTensor(next_state).to(device)
    _, next_value = model(next_state)
    returns = compute_gae(next_value, rewards, masks, values)
    
    log_probs = torch.cat(log_probs)
    returns   = torch.cat(returns).detach()
    values    = torch.cat(values)

    advantage = returns - values

    actor_loss  = -(log_probs * advantage.detach()).mean()
    critic_loss = advantage.pow(2).mean()

    loss = actor_loss + 0.5 * critic_loss - 0.001 * entropy

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

torch.Size([16, 210, 160, 3])


ValueError: cannot reshape array of size 1612800 into shape (210,160,3)

In [ ]:
test_env(True)

In [ ]:
123